In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, count, row_number, when, col, regexp_replace, substring, udf, to_date, avg
from pyspark.sql.functions import round as round_df
from pyspark.sql.window import Window
from pyspark.sql.types import FloatType

import geopy.distance

spark = SparkSession.builder.master("local[*]").appName("databases_project").getOrCreate()
df = spark.read.csv("Crime_Data_from_2010_to_2019.csv", header=True, inferSchema=True)

24/01/10 19:55:24 WARN Utils: Your hostname, MacBook-Air-di-Davide.local resolves to a loopback address: 127.0.0.1; using 192.168.1.192 instead (on interface en0)
24/01/10 19:55:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/10 19:55:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

2. Create a DataFrame that contains the main data-set. Keep the original column names but change the column types as instructed below:
• Date Rptd: date
• DATE OCC: date
• Vict Age: integer
• LAT: double
• LON: double
Print the total number of rows for the entire data-set and the data type of every column.
(5%)

In [3]:
df = df.withColumn("Date Rptd", to_date("Date Rptd", "MM/dd/yyyy hh:mm:ss a"))
df = df.withColumn("DATE OCC", to_date("DATE OCC", "MM/dd/yyyy hh:mm:ss a"))
df = df.withColumn("Vict Age", df["Vict Age"].cast("int"))
df = df.withColumn("LAT", df["LAT"].cast("float"))
df = df.withColumn("LON", df["LON"].cast("float"))
print(df.count())
df.printSchema()

2118997
root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: date (nullable = true)
 |-- DATE OCC: date (nullable = true)
 |-- TIME OCC: integer (nullable = true)
 |-- AREA : integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true)
 |-- Crm Cd 4: integer (nullable = true)
 

3. Implement Query 1 using both the DataFrame and SQL APIs. Execute both implementations with 4 Spark executors. Do you notice differences in the execution times? Justify your answer. (15%)
Find, for each year, the top-3 months with highest number of recorded crimes committed. You are asked to print the month, year, number of criminal acts recorded, as well as the ranking of the month within the respective year. Results are expected to be sorted in ascending order with respect to the year and descending order with respect to the number of crimes (as in Table 1).

In [4]:
# Group by year and month, count the number of crimes
df_grouped = df.groupBy(year("DATE OCC").alias("Year"), month("DATE OCC").alias("Month")) \
               .agg(count("*").alias("crime_total"))

# Rank the months within each year based on the number of crimes
window_spec = Window.partitionBy("Year").orderBy(df_grouped["crime_total"].desc())
df_ranked = df_grouped.withColumn("#", row_number().over(window_spec))

# Filter for the top-3 months with the highest number of crimes for each year
df_top3 = df_ranked.filter(df_ranked["#"] <= 3)

# Sort the results in ascending order with respect to the year and descending order with respect to the number of crimes
df_sorted = df_top3.orderBy("Year", df_top3["crime_total"].desc())

# Print the month, year, number of criminal acts recorded, and ranking of the month within the respective year
df_sorted.select("Month", "Year", "crime_total", "#").show()

+-----+----+-----------+---+
|Month|Year|crime_total|  #|
+-----+----+-----------+---+
|    7|2010|       6037|  1|
|   10|2010|       6035|  2|
|    3|2010|       6032|  3|
|    3|2011|      14953|  1|
|    5|2011|      14896|  2|
|    4|2011|      14396|  3|
|    1|2012|      31423|  1|
|    8|2012|      31041|  2|
|   10|2012|      30921|  3|
|    1|2013|       8691|  1|
|    8|2013|       8008|  2|
|   12|2013|       8001|  3|
|    5|2014|       5296|  1|
|    6|2014|       5248|  2|
|    7|2014|       4830|  3|
|    3|2015|      10200|  1|
|    5|2015|      10018|  2|
|    7|2015|       9785|  3|
|   12|2016|      16670|  1|
|   10|2016|      16616|  2|
+-----+----+-----------+---+
only showing top 20 rows



In [5]:
df.createOrReplaceTempView("df")
sql_df = spark.sql("""
SELECT Year, Month, crime_total, Rank
FROM (
    SELECT Year, Month, crime_total, ROW_NUMBER() OVER (PARTITION BY Year ORDER BY crime_total DESC) AS Rank
    FROM (
        SELECT YEAR(`DATE OCC`) AS Year, MONTH(`DATE OCC`) AS Month, COUNT(*) AS crime_total
        FROM df
        GROUP BY Year, Month
    ) t
) t
WHERE Rank <= 3
ORDER BY Year ASC, crime_total DESC
""")
sql_df.show()


24/01/10 19:55:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-----+-----------+----+
|Year|Month|crime_total|Rank|
+----+-----+-----------+----+
|2010|    7|       6037|   1|
|2010|   10|       6035|   2|
|2010|    3|       6032|   3|
|2011|    3|      14953|   1|
|2011|    5|      14896|   2|
|2011|    4|      14396|   3|
|2012|    1|      31423|   1|
|2012|    8|      31041|   2|
|2012|   10|      30921|   3|
|2013|    1|       8691|   1|
|2013|    8|       8008|   2|
|2013|   12|       8001|   3|
|2014|    5|       5296|   1|
|2014|    6|       5248|   2|
|2014|    7|       4830|   3|
|2015|    3|      10200|   1|
|2015|    5|      10018|   2|
|2015|    7|       9785|   3|
|2016|   12|      16670|   1|
|2016|   10|      16616|   2|
+----+-----+-----------+----+
only showing top 20 rows



Implement Query 2 using both the DataFrame/SQL and RDD APIs. Report and compare execution times for 4 Spark executors. (20%)
Sort the different parts of the day taking into account crimes that were committed on the (STREET), in descending order. Consider the following pars of the day:
• Morning: 5.00am – 11.59am • Afternoon: 12.00pm – 4.59pm
• Evening: 5.00pm – 8.59pm • Night: 9.00pm – 3.59am

In [6]:
# Dividing the "Time OCC" column into 4 parts of the day
df_day_parts = df.withColumn("Day_Part",
                             when((col("Time OCC") >= 500) & (col("Time OCC") < 1200), "Morning")
                             .when((col("Time OCC") >= 1200) & (col("Time OCC") < 1700), "Afternoon")
                             .when((col("Time OCC") >= 1700) & (col("Time OCC") < 2100), "Evening")
                             .otherwise("Night"))

# Group by the day parts and count the number of crimes
df_sorted = df_day_parts.filter(col("Premis Desc") == "STREET").groupBy(col("Day_Part")).agg(count("*").alias("crime_total")).orderBy(col("crime_total").desc())
df_sorted.show()

+---------+-----------+
| Day_Part|crime_total|
+---------+-----------+
|    Night|     169568|
|  Evening|     131422|
|Afternoon|     104408|
|  Morning|      87648|
+---------+-----------+



In [7]:
# Filter the RDD to include only crimes committed on the street
rdd_filtered = df.rdd.filter(lambda row: row["Premis Desc"] == "STREET")

# Map each crime record to a day part
rdd_mapped = rdd_filtered.map(lambda row: (
    "Morning" if 500 <= row["TIME OCC"] < 1200 else
    "Afternoon" if 1200 <= row["TIME OCC"] < 1700 else
    "Evening" if 1700 <= row["TIME OCC"] < 2100 else
    "Night"
)
)

# Map day parts to a tuple containing key and 1, reduce by key to aggregate the counts for each day part
rdd_reduced = rdd_mapped.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

# Sort the resulting RDD in descending order based on the count
rdd_sorted = rdd_reduced.sortBy(lambda x: x[1], ascending=False)

# Print the sorted RDD
for record in rdd_reduced.collect():
    print(record)


('Evening', 131422)
('Morning', 87648)
('Night', 169568)
('Afternoon', 104408)


5. Implement Query 3 using the DataFrame/SQL API. Report and compare execution times for 2, 3 and 4 Spark executors. (20%)
Find the descent of the victims of recorded crimes in Los Angeles for the year 2015 in the 3 ZIP Code areas with the highest and the 3 ZIP Codes with the lowest income per household. Results are expected to be printed from highest to lowest number of victims per ethnic group.

In [8]:
# Reading additional dataframe
income2015_df = spark.read.csv("income/LA_income_2015.csv", header=True, inferSchema=True)
# Converting from string format to integer
income2015_df = income2015_df.withColumn("Estimated Median Income", regexp_replace("Estimated Median Income", "\\$", ""))
income2015_df = income2015_df.withColumn("Estimated Median Income", regexp_replace("Estimated Median Income", ",", "").cast("int"))

In [9]:
# Sorting dataframe by descending order of income
income2015_df_sorted = income2015_df.sort(col("Estimated Median Income").desc())
# Selecting top 3 and bottom 3 rows
highest_rows = income2015_df_sorted.select("Zip Code").head(3)
lowest_rows = income2015_df_sorted.select("Zip Code").tail(3)
highest_zipcodes = [row["Zip Code"] for row in highest_rows]
lowest_zipcodes = [row["Zip Code"] for row in lowest_rows]
zipcodes = highest_zipcodes + lowest_zipcodes

In [10]:
# Reading additional dataframe needed for reverse geocoding
geo_df = spark.read.csv("revgecoding.csv", header=True, inferSchema=True)
geo_df = geo_df.withColumn("ZIPcode", substring(col("ZIPcode"), 0, 5).cast("int"))
geo_df = geo_df.withColumn("LAT", geo_df["LAT"].cast("float"))
geo_df = geo_df.withColumn("LON", geo_df["LON"].cast("float"))

In [11]:
# Filter the DataFrame for the year 2015
df_2015 = df.filter(year("DATE OCC") == 2015)

# Filter the victemless crimes
df_2015 = df_2015.filter(df_2015["Vict Age"] > 0)

# Join the DataFrame with the reverse geocoding DataFrame
df_2015 = df_2015.join(geo_df, on=["LAT", "LON"], how="left")

# Map column "Vict Descent" to required format
df_2015 = df_2015.withColumn("Vict Descent", \
                            when(df_2015["Vict Descent"] == "W", "White") \
                            .when(df_2015["Vict Descent"] == "B", "Black") \
                            .when(df_2015["Vict Descent"] == "H", "Hispanic/Latin/Mexican") \
                            .when(df_2015["Vict Descent"] == "L", "Hispanic/Latin/Mexican") \
                            .when(df_2015["Vict Descent"] == "M", "Hispanic/Latin/Mexican") \
                            .otherwise("Unknown")
)

# Filter the DataFrame for the top-3 and bottom-3 zip codes and both
df_2015_all_zipcodes = df_2015.filter(df_2015["ZIPCode"].isin(zipcodes))
df_2015_high = df_2015.filter(df_2015["ZIPCode"].isin(highest_zipcodes))
df_2015_low = df_2015.filter(df_2015["ZIPCode"].isin(lowest_zipcodes))

# Group by the victim descent and count the number of crimes
df_2015_all_zipcodes.groupBy("Vict Descent").agg(count("*").alias("#")).sort(col("#").desc()).show()
df_2015_high.groupBy("Vict Descent").agg(count("*").alias("#")).sort(col("#").desc()).show()
df_2015_low.groupBy("Vict Descent").agg(count("*").alias("#")).sort(col("#").desc()).show()

# We also did the same thing using geopy but it was too slow
# def get_zipcode(lat, lon):
#     geolocator = Nominatim(user_agent="my_geocoder")
#     location = geolocator.reverse((lat, lon), exactly_one=True)
#     if location:
#         address = location.raw.get("address")
#         if address:
#             return int(address.get("postcode"))
#     return None

# get_zipcode_udf = udf(get_zipcode, IntegerType())

# df_2015 = df_2015.withColumn("Zip Code", get_zipcode_udf(df_2015["LAT"], df_2015["LON"]))

+--------------------+---+
|        Vict Descent|  #|
+--------------------+---+
|Hispanic/Latin/Me...|118|
|               White|102|
|               Black| 99|
|             Unknown| 33|
+--------------------+---+



+------------+---+
|Vict Descent|  #|
+------------+---+
+------------+---+



+--------------------+---+
|        Vict Descent|  #|
+--------------------+---+
|Hispanic/Latin/Me...|118|
|               White|102|
|               Black| 99|
|             Unknown| 33|
+--------------------+---+



Implement Query 4 using the DataFrame/SQL API. (20%)
For the last query, you are asked to examine whether the police stations that respond to recorded crimes in Los Angeles are the closest ones to the crime scene. Towards that end, you are asked to implement and execute two pairs of similar queries and compare results:
- Calculate the number of crimes committed with the use of firearms of any kind and the average distance (in km) of the crime scene to the police station that handled the case. The results should appear ordered by year in ascending order. Additionally, calculate the same stats (number of crimes committed with the use of firearms of any kind and average distance) per police station. Results should appear ordered by number of incidents, in descending order (as in Table 3).
- Calculate the number of crimes committed with the use of firearms of any kind and the average distance (in km) of the crime scene to the police station that is located closest to the crime scene. The results should appear ordered by year in ascending order. Additionally, calculate the same stats (number of crimes committed with the use of firearms of any kind and average distance) per police station. Results should appear ordered by number of incidents, in descending order (as in Table 4).


In [12]:
# Reading additional dataframe containing police stations
police_df = spark.read.csv("LAPD_Police_Stations.csv", header=True, inferSchema=True)
police_df = police_df.withColumn("Police_LON", police_df["X"].cast("float").alias("Police_LON"))
police_df = police_df.withColumn("Police_LAT", police_df["Y"].cast("float").alias("Police_LAT"))
police_df = police_df.drop("X", "Y")
police_df.printSchema()

root
 |-- FID: integer (nullable = true)
 |-- DIVISION: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- PREC: integer (nullable = true)
 |-- Police_LON: float (nullable = true)
 |-- Police_LAT: float (nullable = true)



In [13]:
def get_distance (lat1 , lon1 , lat2 , lon2 ) :
    return round(geopy.distance.distance((lat1, lon1), (lat2, lon2)).km, 3)

# Defining a UDF to calculate distance between two points
get_distance_udf = udf(get_distance, FloatType())

# Filtering values with 0 latitude and longitude 
df_null_island = df.where((col("LAT") != 0) & (col("LON") != 0))
# Filtering values with weapon code between 100 and 200 i.e. guns
df_guns = df_null_island.where((col("Weapon Used Cd") >= 100) & (col("Weapon Used Cd") < 200))
# Joining guns dataframe with police dataframe
df_guns_police = df_guns.join(police_df, [df_guns["AREA "] == police_df["PREC"]], how="left")
# Selecting useful columns
df_crimes = df_guns_police.select("LAT", "LON", "Police_LAT", "Police_LON", "PREC", "DATE OCC", "DIVISION")

# Calculating distance between crime location and police station that responded to the crime
df_crimes = df_crimes.withColumn("Distance", get_distance_udf(df_crimes["LAT"], df_crimes["LON"], df_crimes["Police_LAT"], df_crimes["Police_LON"]))

# Grouping by year and calculating average distance
df_stats1 = df_crimes.groupBy(year("DATE OCC").alias("Year")).agg(count("*").alias("Number of Crimes"), avg("Distance").alias("Average Distance (km)")).orderBy("Year")
# Rounding the average distance to 3 decimal places
df_stats1 = df_stats1.withColumn("Average Distance (km)", round_df(df_stats1["Average Distance (km)"], 3))

# Grouping by division and calculating average distance
df_stats2 = df_crimes.groupBy("DIVISION").agg(count("*").alias("Number of Crimes"), avg("Distance").alias("Average Distance (km)")).orderBy(col("Number of Crimes").desc())
# Rounding the average distance to 3 decimal places
df_stats2 = df_stats2.withColumn("Average Distance (km)", round_df(df_stats2["Average Distance (km)"], 3))

df_stats1.show()
df_stats2.show()

+----+----------------+---------------------+
|Year|Number of Crimes|Average Distance (km)|
+----+----------------+---------------------+
|2010|            2686|                 2.56|
|2011|            5855|                 2.75|
|2012|           12021|                2.836|
|2013|            2271|                2.731|
|2014|            2320|                2.684|
|2015|            3500|                2.653|
|2016|            6537|                2.682|
|2017|            9890|                2.764|
|2018|            2242|                2.565|
|2019|            7129|                2.739|
|2021|           10175|                2.695|
|2022|            9884|                2.709|
|2023|            2820|                2.862|
+----+----------------+---------------------+



+---------------+----------------+---------------------+
|       DIVISION|Number of Crimes|Average Distance (km)|
+---------------+----------------+---------------------+
|    77TH STREET|           12704|                2.677|
|      SOUTHEAST|            8373|                2.097|
|         NEWTON|            7908|                2.019|
|      SOUTHWEST|            6301|                2.689|
|     HOLLENBECK|            4346|                2.719|
|         HARBOR|            3881|                4.077|
|        RAMPART|            3335|                1.631|
|      NORTHEAST|            2967|                3.876|
|        OLYMPIC|            2897|                1.829|
|        MISSION|            2877|                4.666|
|       FOOTHILL|            2618|                3.852|
|      HOLLYWOOD|            2599|                1.447|
|       WILSHIRE|            2472|                2.333|
|NORTH HOLLYWOOD|            2449|                2.741|
|    WEST VALLEY|            22

In [14]:
import pyspark.sql.functions as F

# Repeating the same steps as above but instead the police station with the closest distance is selected
police_cross_join = police_df.crossJoin(df_guns)
# Calculating distance between crime location and closest police station
police_cross_join = police_cross_join.withColumn("Distance", get_distance_udf(police_cross_join["LAT"], police_cross_join["LON"], police_cross_join["Police_LAT"], police_cross_join["Police_LON"]))
window_spec = Window.partitionBy("DR_NO").orderBy(col("Distance").asc())
closest_police_station_df = police_cross_join.withColumn("row_num", F.row_number().over(window_spec)).filter("row_num = 1").drop("row_num")

In [15]:
# Calculating the same stats as above
df_stats11 = closest_police_station_df.groupBy(year("DATE OCC").alias("Year")).agg(count("*").alias("Number of Crimes"), avg("Distance").alias("Average Distance (km)")).orderBy("Year")

df_stats11 = df_stats11.withColumn("Average Distance (km)", round_df(df_stats11["Average Distance (km)"], 3))

df_stats22 = closest_police_station_df.groupBy("DIVISION").agg(count("*").alias("Number of Crimes"), avg("Distance").alias("Average Distance (km)")).orderBy(col("Number of Crimes").desc())

df_stats22 = df_stats22.withColumn("Average Distance (km)", round_df(df_stats22["Average Distance (km)"], 3))

df_stats11.show()
df_stats22.show()

+----+----------------+---------------------+
|Year|Number of Crimes|Average Distance (km)|
+----+----------------+---------------------+
|2010|            2686|                2.271|
|2011|            5579|                2.409|
|2012|            6532|                2.505|
|2013|            2271|                2.415|
|2014|            2320|                 2.16|
|2015|            3500|                 2.46|
|2016|            6076|                2.414|
|2017|            7786|                2.392|
|2018|            2242|                2.348|
|2019|            7129|                2.429|
|2021|            8483|                2.481|
|2022|            6963|                2.404|
|2023|            2820|                2.596|
+----+----------------+---------------------+



+---------------+----------------+---------------------+
|       DIVISION|Number of Crimes|Average Distance (km)|
+---------------+----------------+---------------------+
|    77TH STREET|            7468|                1.717|
|      SOUTHWEST|            7275|                2.228|
|      SOUTHEAST|            6363|                2.206|
|     HOLLENBECK|            4376|                2.703|
|       WILSHIRE|            4256|                 2.46|
|         HARBOR|            3800|                 3.91|
|         NEWTON|            3688|                1.578|
|      HOLLYWOOD|            3550|                 1.97|
|        OLYMPIC|            3137|                1.676|
|        RAMPART|            2913|                1.419|
|       VAN NUYS|            2656|                2.955|
|       FOOTHILL|            2487|                3.616|
|NORTH HOLLYWOOD|            2001|                2.737|
|      NORTHEAST|            1851|                3.736|
|        CENTRAL|            15